# NASA's Mission to Mars

In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# Set executable path and browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/colinbrineman/.wdm/drivers/chromedriver/mac64/98.0.4758.80/chromedriver] found in cache


## Web scraping

### Scrape article from MARS Planet Science

In [3]:
# Visit the Mars NASA news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

**browser.is_element_present_by_css('div.list_text', wait_time=1)**
1. we're searching for elements with a specific combination of: tag (**div**) & attribute (**list_text**). for example, **ul.item_list** would be **\<ul class="item_list">** in html.
2. telling our browser to wait one second before searching for components. optional delay is useful because sometimes dynamic pages take a little while to load, especially if they are image-heavy.

In [4]:
# Set up html parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

**slide_elem**
- **slide_elem** assigned as variable to look for the **<div \/>** tag and its "descendent" (i.e., the other tags within the **<div \/> element**)?
- **slide_elem** is our "parent element"
- a parent elements element holds all of the other elements within it
- we'll reference the parent element later when we want to filter search results even further

dot (".")
- used for selecting classes, such as **list_text**
- the code **div.list_text** pinpoints the **<div \/>** tag with the class of **list_text**

**select_one**
- CSS works "from right to left"
- i.e., it returns the last item on the list instead of the first
- when using **select_one**, the first matching element returned will be a **<li \/>** element with a class of **slide** (and all nested elements within it)

In [5]:
# Use the parent element to find 1st "a" tag & save it as "news_title"
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

'Join NASA for the Launch of the Mars 2020 Perseverance Rover'

chaining **.get_text()** onto **.find()** onto **slide_elem**
- **.find()** is used when we want only the first class and attribute we've specified
- we're saying, "the variable **slide_elem** holds a ton of information, so look inside of that information to find this specific data"
- we've specified that we're looking for the content title by saying, "The specific data is in a **<div \/>** with a class of **content_title**
- **.get_text()** means we only want the text (no html code within tags)

In [6]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'No matter where you live, choose from a menu of activities to join NASA as we "Countdown to Mars" and launch the Perseverance rover to the Red Planet.'

### Scrape image from Jet Propulsion Laboratory

In [7]:
# Set executable path and browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/colinbrineman/.wdm/drivers/chromedriver/mac64/98.0.4758.80/chromedriver] found in cache


In [8]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [9]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

indexing
- chained at the end of the first line of code
- stipulates that we want our browser to click the 2nd button
.click()
- Splinter method applied to variable to find button
- tells Splinter to "click" the button

In [10]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [11]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

**.get('src')**
- we're telling BeautifulSoup to look inside the **<img \/>** tag for an image with a class of **fancybox-image**
- **.get('src')** pulls the link to the image (designated by "src" in the tag)

In [12]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

### Scrape data from Mars Facts

In [13]:
# Scrape table with Pandas
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


**df = pd.read_html('https://galaxyfacts-mars.com')[0]**
- creates a new DataFrame from the html table
- the Pandas function **read_html()** specifically searches for and returns a list of tables found in the html
- by specifying an index of "0," we're telling Pandas to pull only the first table it encounters, or the first item in the list
- then, it turns the table into a DataFrame.

**df.columns=['description', 'Mars', 'Earth']**
- assigns columns to the new DataFrame for additional clarity.

**df.set_index('description', inplace=True)**
- by using the **.set_index()** function, we're turning the "Description" column into the DataFrame's index
- **inplace=True** means that the updated index will remain in place, without having to reassign the DataFrame to a new variable

In [14]:
# Convert DataFrame back into html-ready code
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [15]:
# End session
browser.quit()